In [1]:
import nltk 
import pandas as pd
import numpy as np
import re
import sklearn
import tensorflow as tf

In [2]:
f=open("/home/prayalankar/data/train_1000.label","r+b")
col=['category','label','data']
df=pd.DataFrame(columns=col)
while True:
    as1=f.readline()
    if as1=='':
        break
    as1=nltk.word_tokenize(as1)
    str1=' '.join(as1[3:])
    df.loc[len(df)]=[as1[0],as1[2],str1]

In [3]:
def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [4]:
# extractinga and cleaning the text data
x=df.data
t_data=[]
for p in x:
    t_data.append(p)
t_data=[clean_str(p) for p in t_data]
# onehot encoding of target
number=sklearn.preprocessing.LabelEncoder()
df['col3']=number.fit_transform(df['category'])
ye=df.col3

In [5]:
#making an average vector of the data in each row
def genfeat(words):
    featureVec=np.zeros(50,)
    nwords=0
    for word in words:
        if word in dict_w2v.keys(): 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,dict_w2v[word])
            featureVec =np.divide(featureVec , nwords)
    return(featureVec)

In [6]:
# vector values of words using glove.6B pretrained word2vec
dict_w2v = {}
all_words = set(w for words in t_data for w in nltk.word_tokenize(words))
with open("/home/prayalankar/gloveb/p6B50d.txt", "rb") as ff:
    for line in ff:
        parts = line.split()
        word = parts[0]
        nums = map(float, parts[1:])
        if word in all_words:
            dict_w2v[word] = np.array(nums)

In [7]:
#making the input sequences 
x_text=[]
for i in range(len(t_data)):
    a_temp=genfeat(t_data[i])
    x_text.append(a_temp)


In [8]:
t_input  = []
for i in x_text:
    temp_list = []
    for j in i:
            temp_list.append([j])
    t_input.append(np.array(temp_list))
train_input = t_input

In [9]:
train_output=[]
for i in ye:
    temp_list = ([0]*6)
    temp_list[i]=1
    train_output.append(temp_list)


In [10]:
test_input = train_input[900:]
test_output = train_output[900:]
train_input = train_input[:900]
train_output = train_output[:900]

In [11]:
#defining placeholders for input/output
data = tf.placeholder(tf.float32, [None, 50,1]) #Number of examples, number of input, dimension of each input
label = tf.placeholder(tf.float32, [None, 6])
hidden_layers = 12
#defining cells
cell = tf.nn.rnn_cell.LSTMCell(hidden_layers,state_is_tuple=True)
val, _ = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [12]:
val = tf.transpose(val, [1, 0, 2])
l_last = tf.gather(val, int(val.get_shape()[0]) - 1)

#defining weights and biases
w_weight = tf.Variable(tf.truncated_normal([hidden_layers, int(label.get_shape()[1])]))
b_bias = tf.Variable(tf.constant(0.1, shape=[label.get_shape()[1]]))

#prediction using neuralnet softmax algo on top of lstm cells
p_prediction = tf.nn.softmax(tf.matmul(l_last, w_weight) + b_bias)

#cross entropy for multiclass classification
cross_entropy = -tf.reduce_sum(label * tf.log(p_prediction))


optimizer = tf.train.AdamOptimizer()
rfunc = optimizer.minimize(cross_entropy)
m_mis = tf.not_equal(tf.argmax(label, 1), tf.argmax(p_prediction, 1))
error = tf.reduce_mean(tf.cast(m_mis, tf.float32))

/home/prayalankar/anaconda3/envs/bootr/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

b_size = 90
no_batches = int(len(train_input)) / b_size
niter = 300
for i in range(niter):
    pos = 0
    for j in range(no_batches):
        inp, out = train_input[pos:pos+b_size], train_output[pos:pos+b_size]
        pos+=b_size
        sess.run(rfunc,{data: inp, label: out})
    print ("iter_no",str(i) )
e_rate = sess.run(error,{data: test_input, label: test_output})
apple=[]
for i in range(len(test_input)):
    apple.append(sess.run(p_prediction,{data:[test_input[i] ] }) )
print('iter_no {:2d} error {:3.1f}%'.format(i + 1, 100 * e_rate))
sess.close()

('iter_no', '0')
('iter_no', '1')
('iter_no', '2')
('iter_no', '3')
('iter_no', '4')
('iter_no', '5')
('iter_no', '6')
('iter_no', '7')
('iter_no', '8')
('iter_no', '9')
('iter_no', '10')
('iter_no', '11')
('iter_no', '12')
('iter_no', '13')
('iter_no', '14')
('iter_no', '15')
('iter_no', '16')
('iter_no', '17')
('iter_no', '18')
('iter_no', '19')
('iter_no', '20')
('iter_no', '21')
('iter_no', '22')
('iter_no', '23')
('iter_no', '24')
('iter_no', '25')
('iter_no', '26')
('iter_no', '27')
('iter_no', '28')
('iter_no', '29')
('iter_no', '30')
('iter_no', '31')
('iter_no', '32')
('iter_no', '33')
('iter_no', '34')
('iter_no', '35')
('iter_no', '36')
('iter_no', '37')
('iter_no', '38')
('iter_no', '39')
('iter_no', '40')
('iter_no', '41')
('iter_no', '42')
('iter_no', '43')
('iter_no', '44')
('iter_no', '45')
('iter_no', '46')
('iter_no', '47')
('iter_no', '48')
('iter_no', '49')
('iter_no', '50')
('iter_no', '51')
('iter_no', '52')
('iter_no', '53')
('iter_no', '54')
('iter_no', '55')
('

In [14]:
apple

[array([[ 0.00733837,  0.13255183,  0.28398651,  0.2644318 ,  0.16406678,
          0.14762478]], dtype=float32),
 array([[ 0.01889388,  0.18440688,  0.24399821,  0.1736083 ,  0.19312128,
          0.18597133]], dtype=float32),
 array([[ 0.01107893,  0.15029271,  0.269077  ,  0.21640067,  0.18458593,
          0.16856483]], dtype=float32),
 array([[ 0.00675094,  0.12923411,  0.28668684,  0.27523753,  0.15908374,
          0.14300686]], dtype=float32),
 array([[ 0.01105419,  0.15022285,  0.26914495,  0.21667495,  0.18446456,
          0.16843852]], dtype=float32),
 array([[ 0.00443684,  0.11319427,  0.29903057,  0.33286297,  0.13108131,
          0.11939406]], dtype=float32),
 array([[ 0.00556691,  0.12174872,  0.29255629,  0.30114219,  0.14680833,
          0.13217762]], dtype=float32),
 array([[ 0.00542279,  0.1207502 ,  0.29331613,  0.30474553,  0.14506388,
          0.13070151]], dtype=float32),
 array([[ 0.00408573,  0.11221427,  0.305796  ,  0.35742784,  0.11257143,
          0.10

In [21]:
i,j = np.unravel_index(apple[87].argmax(), apple[87].shape)

IndexError: list index out of range